In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from numpy.fft import fft2, ifft2, fftshift, ifftshift

In [2]:
# Read the uploaded image

image_path = './pisicaaaa.png'
image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)


image_orig = image.copy()
kernel_size = (21, 21)
sigma = 21

blurred_image_kernel = cv2.GaussianBlur(image, kernel_size, sigma)

f_transform = fftshift(fft2(image))


def generate_2d_gaussian_kernel(height, width, sigma):
    kernel = np.zeros((height, width))
    center = (height // 2, width // 2)
    for i in range(height):
        for j in range(width):
            kernel[i, j] = np.exp(
                -((i - center[0]) ** 2 + (j - center[1]) ** 2) / (2 * sigma**2)
            )

    return kernel

mask = generate_2d_gaussian_kernel(image.shape[0], image.shape[1], sigma)


f_transform_masked = f_transform * mask
blurred_image_fft = np.abs(np.real(ifft2(ifftshift(f_transform_masked)))).astype(np.uint8)


def get_fft(img):
    image = 10 * np.log10(np.abs(np.real(fftshift(fft2(img)))) + 1e-10)

    real_part_normalized = (image - np.min(image)) / (np.max(image) - np.min(image))
    real_part_scaled = (real_part_normalized * 255).astype(np.uint8)
    return real_part_scaled

fig, axes = plt.subplots(2, 3, figsize=(18, 12))

axes[0, 0].imshow(image_orig, cmap='gray')
axes[0, 0].set_title('Original Image')

axes[0, 1].imshow(blurred_image_kernel, cmap='gray')
axes[0, 1].set_title('Blurred Image with Kernel')

axes[0, 2].imshow(blurred_image_fft, cmap='gray')
axes[0, 2].set_title('Blurred Image with FFT')

axes[1, 0].imshow(get_fft(image_orig), cmap='gray')
axes[1, 0].set_title('Spectrum of Original Image')

axes[1, 1].imshow(get_fft(blurred_image_kernel), cmap='gray')
axes[1, 1].set_title('Spectrum Blur with Kernel')

axes[1, 2].imshow(get_fft(blurred_image_fft), cmap='gray')
axes[1, 2].set_title('Spectrum Blurr with FFT')


plt.tight_layout()
plt.show()